# Importing neccesary python modules

In [54]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from datetime import timedelta
import tensorflow as tf

# Importing Dataset

In [55]:
# To Import Data from google drive (authentication needed)
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd 
Data=pd.read_csv('gdrive/My Drive/Personal Data Science Projects/Apple Stocks Predictions/AAPL.csv', parse_dates=['Date'])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###  Droppping Past Stocks data ( Data on year 2017 and less )


In [56]:
# Data.head()
# Data = Data[ Data['Date']>=datetime(2016,1,1) ]

### Renaming the Data Columns


In [57]:
Data['date'] = Data[['Date']]
Data['close'] = Data['Close']
df = Data.copy()

# Visualization of the Data ( Close_Price vs Date )


In [58]:
fig = px.line( x = Data['Date'],y=Data['Close'] )
fig.update_layout( yaxis_title='Close Price' )
fig.show()

*** 
***

# Univariate Time Series Predictions using LTSM 

# Scaling the Close Price column using StandardScaler


In [59]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit( df[['close']] )
df['close'] = scaler.transform( df[['close']] )
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,date,close
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.535085,493729600,2010-01-04,-0.902807
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.546383,601904800,2010-01-05,-0.902502
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.442257,552160000,2010-01-06,-0.905311
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.430347,477131200,2010-01-07,-0.905632
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.473095,447610800,2010-01-08,-0.904479


In [60]:
df.shape

(3126, 9)

# Defining a function to create Timestaped Data Matrix in which window size can be given as input


In [61]:
def Create_Timestaped_Data_Matrix_Uni( df, size=50 ):
  X,y,dates = [],[],[]
  for i in range(df.shape[0]-size):
    X.append( df['close'].values[i:i+size] )
    y.append( df['close'].values[i+size] )
    dates.append(df['date'][i+size])
  return np.array(X), np.array(y), dates

window_size = 100
X,y,dates = Create_Timestaped_Data_Matrix_Uni( df, window_size )

# Train-Test-Split

In [62]:
test_size_ratio = 0.1
train_size = int((1-test_size_ratio)*len(y))
# Train Set
y_train = y[:train_size]
dates_train = dates[:train_size]
y_test = y[train_size:]
X_train = X[:train_size] 
# Test Set
dates_test = dates[train_size:]
X_test = X[train_size:]
y_train_original = Data['close'][window_size:train_size+window_size] # Storing original columns for plotting graph
y_test_original = Data['close'][train_size+window_size:] # Storing original columns for plotting graph
# Check length
# len(y_train)+len(y_test),len(y)

In [63]:
## Splitting data according a particular date ( "split_date" )
# import datetime as dt
# split_date = df['date'][1100]
# split_date

# test = Data[ Data['date']>=split_date]
# train = Data[ Data['date']<Jan2019 ]
# test_date = Data[ Data['date']>=Jan2019].values
# train_date = Data[ Data['date']<Jan2019 ].values
# train.shape, test.shape

# X_train, y_train = Create_Timestaped_Data_Matrix( train, size=50 )
# X_test, y_test = Create_Timestaped_Data_Matrix( test, size=50 )

### Graph of Splitted Data

In [64]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train,mode='lines', name="Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test,mode='lines', name="Test Set" ) )
fig.show()

### Reshaping and converting splited data into Dataframe for next step



In [65]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
X_train = pd.DataFrame( X_train )
X_test = pd.DataFrame( X_test )

# Applying LSTM Model

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [67]:
model = Sequential()
model.add( LSTM( 500, return_sequences =True, input_shape=(100,1) ) )
model.add( LSTM(100) ) 
model.add( Dense(1) )
model.compile( loss='mean_squared_error', optimizer='adam' )
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 500)          1004000   
                                                                 
 lstm_5 (LSTM)               (None, 100)               240400    
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,244,501
Trainable params: 1,244,501
Non-trainable params: 0
_________________________________________________________________


In [68]:
history = model.fit(X_train,y_train,validation_data = (X_test,y_test), epochs =20,verbose=1)

Epoch 1/20
86/86 [==============================] - 5s 29ms/step - loss: 0.0133 - val_loss: 0.0493
Epoch 2/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0019 - val_loss: 0.0451
Epoch 3/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0019 - val_loss: 0.1035
Epoch 4/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0020 - val_loss: 0.0178
Epoch 5/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0016 - val_loss: 0.0171
Epoch 6/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0015 - val_loss: 0.0495
Epoch 7/20
86/86 [==============================] - 2s 21ms/step - loss: 0.0013 - val_loss: 0.0117
Epoch 8/20
86/86 [==============================] - 2s 22ms/step - loss: 0.0012 - val_loss: 0.0140
Epoch 9/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0012 - val_loss: 0.0140
Epoch 10/20
86/86 [==============================] - 2s 20ms/step - loss: 0.0013 - val_loss: 0.0156
Epoch 11/

In [69]:
model.save('/content/APPL_univariate_lstm_train_test_model.h5')

###  Fucntion defined to change the shape of  Data ( from reshape(-1,1) to   ) 


In [70]:
def changing_shape( array ):
  temp = []
  for i in range(len(array)):
    temp.append(array[i][0])
  return temp

### Predicting Values on Training and Test Data
### Applying inverse scaling then reshaping ( from reshape(-1,1) to (1,:) )

In [71]:
def Prediction_Inverse_Scaling_Reshaping( model, X_train, X_test ):
  # Predicting values and Appyling Inverse Scaler
  y_pred = model.predict( X_test )
  y_pred = scaler.inverse_transform( y_pred )
  y_pred_train = model.predict( X_train )
  y_pred_train = scaler.inverse_transform( y_pred_train )

  # Reshaping
  y_pred = changing_shape(y_pred)
  y_pred_train = changing_shape( y_pred_train )

  return y_pred, y_pred_train


y_pred, y_pred_train = Prediction_Inverse_Scaling_Reshaping( model, X_train, X_test )

# Plotting original data alongwith predicted data

In [72]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train_original,mode='lines', name="Original Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
fig.add_trace( go.Scatter( x=dates_train, y=y_pred_train, name="Predicted Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_pred,mode='lines', name="Predicted Test Set" ) )
fig.show()

* Predictions made by univaite LTSM model seema to be very accurate on training set as well as on test data
* Predicted values follows the values of original days with difference of 1 day(unlike smoothing methods where difference is of several days)

In [73]:
from sklearn.metrics import mean_squared_error
np.sqrt( mean_squared_error( y_pred, y_test_original ) )

3.3102226854417744

***

# Predictions for future dates

### Concatinating Train and Test set

In [74]:
df = df[['date','close']]
X_future = pd.concat( [X_train, X_test], axis=0 )
X_future.reset_index(drop=True,inplace=True)
X_future.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.902807,-0.902502,-0.905311,-0.905632,-0.904479,-0.906020,-0.907988,-0.905575,-0.906580,-0.909463,...,-0.866270,-0.870010,-0.869681,-0.871213,-0.874525,-0.883241,-0.879484,-0.875826,-0.877095,-0.878009
1,-0.902502,-0.905311,-0.905632,-0.904479,-0.906020,-0.907988,-0.905575,-0.906580,-0.909463,-0.901958,...,-0.870010,-0.869681,-0.871213,-0.874525,-0.883241,-0.879484,-0.875826,-0.877095,-0.878009,-0.870397
2,-0.905311,-0.905632,-0.904479,-0.906020,-0.907988,-0.905575,-0.906580,-0.909463,-0.901958,-0.904685,...,-0.869681,-0.871213,-0.874525,-0.883241,-0.879484,-0.875826,-0.877095,-0.878009,-0.870397,-0.867489
3,-0.905632,-0.904479,-0.906020,-0.907988,-0.905575,-0.906580,-0.909463,-0.901958,-0.904685,-0.907700,...,-0.871213,-0.874525,-0.883241,-0.879484,-0.875826,-0.877095,-0.878009,-0.870397,-0.867489,-0.864235
4,-0.904479,-0.906020,-0.907988,-0.905575,-0.906580,-0.909463,-0.901958,-0.904685,-0.907700,-0.916202,...,-0.874525,-0.883241,-0.879484,-0.875826,-0.877095,-0.878009,-0.870397,-0.867489,-0.864235,-0.861665


In [75]:
final_model = Sequential()
final_model.add( LSTM( 500, return_sequences =True, input_shape=(100,1) ) )
final_model.add( LSTM(100) ) 
final_model.add( Dense(1) )
final_model.compile( loss='mean_squared_error', optimizer='adam' )
final_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 100, 500)          1004000   
                                                                 
 lstm_7 (LSTM)               (None, 100)               240400    
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,244,501
Trainable params: 1,244,501
Non-trainable params: 0
_________________________________________________________________


In [76]:
history = final_model.fit( X, y, epochs =20, verbose=1)

Epoch 1/20
95/95 [==============================] - 4s 20ms/step - loss: 0.0277
Epoch 2/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0033
Epoch 3/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0046
Epoch 4/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0033
Epoch 5/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0023
Epoch 6/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0030
Epoch 7/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0023
Epoch 8/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0021
Epoch 9/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0029
Epoch 10/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0017
Epoch 11/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0018
Epoch 12/20
95/95 [==============================] - 2s 18ms/step - loss: 0.0022
Epoch 13/20
95/95 [==================

In [77]:
final_model.save('/content/APPL_univariate_lstm_final_model.h5')

In [79]:
y_pred = final_model.predict( X )
y_pred = scaler.inverse_transform( y_pred )
y_pred = changing_shape(y_pred)

y_pred_test = y_pred[len(y_pred)-len(dates_test):-1]

In [80]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train_original,mode='lines', name="Original Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
fig.add_trace( go.Scatter( x=dates, y=y_pred, name="Predicted Set" ) )
fig.show()

### Preparing Training Set for Predictions 

In [81]:
no_of_future_dates = 15
future_dates = []
for i in range(1,no_of_future_dates+1):
  # print( "day: ", i )

  # Extracting last values values (of certain window size) from the df to use it for predicting stock price for next date
  last = df['close'].iloc[-window_size:].values 
  pred_close = final_model.predict( last.reshape( 1, window_size, 1 ) )[0][0]

  # Adding one day to last date ( for adding "next" day date )
  temp_date = df['date'].iloc[-1] + timedelta(days=1) 
  future_dates.append( temp_date )

  # Creating dataframe of 'next' and prediction on that day then appending them to df
  temp =pd.DataFrame({'date':[temp_date],'close':[pred_close]})
  df = pd.concat( [df,temp], axis=0 )
  df.reset_index(drop=True,inplace=True) 

In [82]:
# Taking Inverse Transform ( same as before )
temp = scaler.inverse_transform( df[['close']].iloc[-no_of_future_dates:] )
# Reshaping 
temp = changing_shape(temp)

future_close_price = temp[-no_of_future_dates:]
future_pred_df = pd.DataFrame( future_dates, columns=['future_dates'] )
future_pred_df['future_close_price'] = future_close_price
future_pred_df

,future_dates,future_close_price
0,2022-06-03,151.542830
1,2022-06-04,152.468355
2,2022-06-05,153.365642
3,2022-06-06,154.252263
4,2022-06-07,155.124961
5,2022-06-08,155.978838
6,2022-06-09,156.811391
7,2022-06-10,157.622136
8,2022-06-11,158.411672
9,2022-06-12,159.180929


In [83]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_train, y=y_train_original,mode='lines', name="Original Train Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
fig.add_trace( go.Scatter( x=dates, y=y_pred, name="Predicted Set" ) )
fig.add_trace( go.Scatter( x=future_dates , y=future_close_price ,mode='lines', name="Future Predictions" ) )
fig.show()

In [84]:
fig = go.Figure()
fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
fig.add_trace( go.Scatter( x=dates_test, y=y_pred_test,mode='lines', name="Predicted Test Set" ) )
fig.add_trace( go.Scatter( x=future_dates , y=future_close_price ,mode='lines', name="Future Predictions" ) )
fig.show()

###### Predictions using Univariate Model ends here
***
***

##### Hypertuning LSTM ---> **Hypertuning** didn't helped in imporving accuracy

In [85]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dropout
# ! pip install keras_tuner
# import keras_tuner as kt
# from tensorflow.keras.models import save_model

# def hypertuning_model( hp ):
#   model = Sequential(  )
#   model.add( LSTM( hp.Int( 'lstm_input', min_value=5, max_value=55, step=10 ), input_shape=(50,1) ,return_sequences=True ) )
#   for i in range( hp.Int('no_of_layers', min_value=1,max_value=4) ):
#     model.add( LSTM( hp.Int( f'lstm_units_{i}', min_value=5, max_value=55, step=10 ), return_sequences=True ) )
#   model.add( LSTM( hp.Int( 'lstm_units', min_value=5, max_value=55, step=10 ), return_sequences=False ) )
#   model.add( Dropout( hp.Choice('rate',[0.0,0.1,0.2,0.3]) ) )
#   model.add( Dense(1, activation='relu' ))
#   model.compile( optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()] )
#   return model

# hyper_tuner_model = kt.Hyperband( hypertuning_model, objective='val_loss', max_epochs=20, factor=3 ,seed=10 )
# hyper_tuner_model.search( X_train, y_train, validation_data=(X_test, y_test), epochs=15 )

# best_model = hyper_tuner_model.get_best_models(num_models=1)[0]
# save_model( best_model,'gdrive/My Drive/Personal Data Science Projects/Apple Stocks Predictions/AppleStocksPredictions_Univariate_LTSM.h5')

# # Get the optimal hyperparameters
# best_hps=hyper_tuner_model.get_best_hyperparameters(num_trials=1)[0]
# best_hps.values

# best_model.summary()

# y_pred, y_pred_train = Prediction_Inverse_Scaling_Reshaping( best_model, X_train, X_test )

# fig = go.Figure()
# fig.add_trace( go.Scatter( x=dates_train, y=y_train_original,mode='lines', name="Original Train Set" ) )
# fig.add_trace( go.Scatter( x=dates_test, y=y_test_original,mode='lines', name="Original Test Set" ) )
# fig.add_trace( go.Scatter( x=dates_train, y=y_pred_train, name="Predicted Train Set" ) )
# fig.add_trace( go.Scatter( x=dates_test, y=y_pred,mode='lines', name="Predicted Test Set" ) )
# fig.show()